In [1]:
from numpy.fft import fft, ifft, hfft
from scipy.signal import lfilter
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy import signal
import import_ipynb
import Helpers

figsize=(10,5)

importing Jupyter notebook from Helpers.ipynb


# The building blocks of a reverberator

In order to build a reverberator, let's first have a look at the components of a reverberator. The first important element to build a reverb is a tap delay, an effect that simply delays copies of the signal with different amplitudes, and sum them up together.

In [ ]:
def tap_delay(x, delay_IR, pole=False):
    
    delays = 0
    
    for i in range(delay_IR.size):
        delays += delay_IR[i] * np.concatenate((np.full((i), 0.), x))[:x.size]
        
    if pole==True:
        b = delay_IR
        a = np.ones(1)
        freq_response(b,a)
        return zplane(b,a)
    
    return delays 

The tap delay being an FIR filter, it is always stable and do not have any pole. It features as many zeros as the size of the impulse response. Let's try to create a tap delay with an impulse reponse of size 5 and observe the corresponging pole zero plot.

In [ ]:
tap_delay(np.array([1]), np.array([1, 0, 0, .5, .4, 0]), pole=True)

The second element we need to create a reverb is the comb filter, or IIR delay. As seen in class, it is a purley recursive filter and hence contains only poles. One must hence be careful to contain the poles inside of the unit circle for the filter to be usable. Given the pole arrangement, the frequency response is a sinusoid.

In [ ]:
def comb_filter(x, g, M, pole=False):
    
    b = np.repeat(np.conj(g),1)
    a = np.zeros(M+1)
    a[0] = 1
    a[M] = g
    
    if pole==True:
        freq_response(b,a)
        return zplane(b,a)
    
    x = lfilter(b, a, x)
    return x / np.max(np.abs(x))

In [ ]:
comb_filter(np.array([1]), .4, 5, pole=True)

The last element needed to create a reverb is the all-pass comb filter. This filter simply cancels the effects of the poles with zeros in order to make the frequency response flat.

In [ ]:
def allpass_comb_filter(x, g, M, pole=False):
    
    b = np.zeros(M+1)
    b[0] = np.conj(g)
    b[M] = 1
    a = np.zeros(M+1)
    a[0] = 1
    a[M] = g
    
    if pole==True:
        freq_response(b,a)
        return zplane(b,a)
    
    x = lfilter(b, a, x)
    return x / np.max(np.abs(x))

In [ ]:
allpass_comb_filter(np.array([1]), .4, 5, pole=True)

We can now compare the impulse responses of the 3 filters.

In [ ]:
impulse = np.zeros(100)
impulse[1] = 1

plt.figure(figsize=(12,6))
plt.stem(impulse, label = "Unit impulse", linefmt='blue', markerfmt='b.',use_line_collection=True)
plt.stem( tap_delay(impulse, np.array([1, 0, 0, .5, .4])),  linefmt='y', label="Tap delay IR", markerfmt='y.',use_line_collection=True)
plt.stem( comb_filter(impulse, 0.6, 10), label="Comb filter IR", linefmt='red', markerfmt='r.',use_line_collection=True)
plt.stem( allpass_comb_filter(impulse, 0.6, 10), label="Allpass comb filter IR", linefmt='g', markerfmt="g.", use_line_collection=True)
plt.legend()
plt.show()

# Schroeder's reverberator

Let's now use the building brick to create actual reverberators, as they were presented in class. The Schoeder reverb startes with parallel comb filters and then series allpass filters.

In [ ]:
def schroeder_reverberator(x):
    
    # Parallel stage
    x = ( comb_filter(x, 0.805, 1801) 
        + comb_filter(x, 0.827, 1478) 
        + comb_filter(x, 0.783, 2011) 
        + comb_filter(x, 0.764, 2123) ) / 4
    
    # Series stage
    x = allpass_comb_filter(x, 0.7, 225)
    x = allpass_comb_filter(x, 0.7, 82)
    x = allpass_comb_filter(x, 0.7, 22)
    
    return x

# Moorer's reverberator

The Moorer reverb instead first uses a tap delay to create early reflections, and then uses a delayed parallel comb and series allpass to create late reverberations. The sum of both give the complete reverb.

In [ ]:
def moorer_reverberator(x):

    # Tap delay
    delay_array = np.zeros(150)
    delay_array[40]  = .4
    delay_array[70]  = .3
    delay_array[149] = .2
    x = tap_delay(x, delay_array)
    early_reflections = x
    
    # Parallel stage
    x = ( comb_filter(x, 0.805, 1801) 
        + comb_filter(x, 0.827, 1478) 
        + comb_filter(x, 0.783, 2011) 
        + comb_filter(x, 0.764, 2123) ) / 4
    
    # Allpass stage
    x = allpass_comb_filter(x, 0.7, 225)
    
    # Delay
    late_reverb = np.concatenate((np.full(1000, 0.), x))[:x.size]
    
    return early_reflections + late_reverb

We can plot the impulse response of both filters.

In [ ]:
impulse = np.zeros(20000)
impulse[1] = 1

plt.figure(figsize=(12,6))
plt.stem(impulse, label = "Unit impulse",  linefmt='blue', markerfmt='b.',use_line_collection=True)
#plt.stem( schroeder_reverberator(impulse),  linefmt='y', label="Schoeder IR", markerfmt='y.',use_line_collection=True)
plt.stem( moorer_reverberator(impulse), label="Moorer IR", linefmt='red', markerfmt='r.',use_line_collection=True)
plt.legend()
plt.show()

plt.figure(figsize=(12,6))
plt.stem(impulse, label = "Unit impulse",  linefmt='blue', markerfmt='b.',use_line_collection=True)
plt.stem( schroeder_reverberator(impulse),  linefmt='y', label="Schoeder IR", markerfmt='y.',use_line_collection=True)
#plt.stem( moorer_reverberator(impulse), label="Moorer IR", linefmt='red', markerfmt='r.',use_line_collection=True)
plt.legend()
plt.show()

# Testing with guitar sample

Time to test out our reverbs with an actual sample of a guitar!

In [ ]:
fs, data = wavfile.read('samples/guitar.wav')
guitar_sample = np.array(data, dtype=np.float32)
guitar_sample /= np.max(np.abs(guitar_sample) )
plt.figure(figsize=figsize)
plt.plot(guitar_sample)
Audio("samples/guitar.wav", autoplay=False)

In [ ]:
guitar_schroeder = schroeder_reverberator(guitar_sample)
guitar_schroeder /= np.max(np.abs(guitar_schroeder) )
plt.figure(figsize=figsize)
plt.plot(guitar_schroeder)
wavfile.write('samples/guitar_schroeder.wav', fs, (0x7FFF * guitar_schroeder).astype(np.int16))
Audio("samples/guitar_schroeder.wav", autoplay=False)

In [ ]:
guitar_moorer = moorer_reverberator(guitar_sample)
guitar_moorer /= np.max(np.abs(guitar_moorer) )
plt.figure(figsize=figsize)
plt.plot(guitar_moorer)
wavfile.write('samples/guitar_moorer.wav', fs, (0x7FFF * guitar_moorer).astype(np.int16))
Audio("samples/guitar_moorer.wav", autoplay=False)